In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

#PATH = "C:/Users/cahib/Documents/Code/CPS_Report_Card/SQRP_Ratings"
PATH = 'C:/Users/sronkowski/Documents/GitHub/CPS_Report_Card/SQRP_Ratings'
OUTPUT_PATH = 'C:/Users/sronkowski/Documents/GitHub/CPS_Report_Card/SQL'

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

## Data Imports

To begin, we will combine the contents the CPS Report Cards from 2015-16 through 2019-2020.  We begin with the first school year under analysis, the 2015-16 school year.

In [3]:
#set sheet name and input document names
sheet_name = 'Elem Schools (grds PreK-8 only)'
sy15_16_doc = 'SY15_SQRP_Report_CPSEDU_FINAL_20151023.xlsx'
sy16_17_doc = 'Accountability_SQRPratings_2016-2017_SchoolLevel.xls'
sy17_18_doc = 'Accountability_SQRPratings_2017-2018_SchoolLevel.xls'
sy18_19_doc = 'Accountability_SQRPratings_2018-2019_SchoolLevel.xls'
sy19_20_doc = 'Accountability_SQRPratings_2019-2020_SchoolLevel_v20200305.xls'

#load first file
df_15_16 = pd.read_excel(f'{PATH}/{sy15_16_doc}',sheet_name = sheet_name, header = [0,1,2], na_values ='.') 

In [4]:
df_15_16.tail().T

476  \
School ID                                          Unnamed: 0_level_1 Unnamed: 0_level_2                            609977   
School Name                                        Unnamed: 1_level_1 Unnamed: 1_level_2                          WOODLAWN   
Network                                            Unnamed: 2_level_1 Unnamed: 2_level_2                         NETWORK 9   
SQRP Total Points Earned                           Unnamed: 3_level_1 Unnamed: 3_level_2                               2.6   
SY 2015-2016 SQRP Rating                           Unnamed: 4_level_1 Unnamed: 4_level_2                           Level 2   
...                                                                                                                    ...   
2015-2016 SQRP Individual Indicator Scores (Bas... Other Indicators   My Voice, My School 5 Essentials Survey.1          5   
                                                                      My Voice, My School 5 Essentials Survey.2         10   
                                                                      Data Quality Index Score                         100   
                                                                      Data Quality Index Score.1                         5   
                                                                      Data Quality Index Score.2                         5   

                                                                                                                       477  \
School ID                                          Unnamed: 0_level_1 Unnamed: 0_level_2                            610345   
School Name                                        Unnamed: 1_level_1 Unnamed: 1_level_2                           WOODSON   
Network                                            Unnamed: 2_level_1 Unnamed: 2_level_2                         NETWORK 9   
SQRP Total Points Earned                           Unnamed: 3_level_1 Unnamed: 3_level_2                                 2   
SY 2015-2016 SQRP Rating                           Unnamed: 4_level_1 Unnamed: 4_level_2                           Level 2   
...                                                                                                                    ...   
2015-2016 SQRP Individual Indicator Scores (Bas... Other Indicators   My Voice, My School 5 Essentials Survey.1          4   
                                                                      My Voice, My School 5 Essentials Survey.2         10   
                                                                      Data Quality Index Score                        99.5   
                                                                      Data Quality Index Score.1                         5   
                                                                      Data Quality Index Score.2                         5   

                                                                                                                       478  \
School ID                                          Unnamed: 0_level_1 Unnamed: 0_level_2                            610234   
School Name                                        Unnamed: 1_level_1 Unnamed: 1_level_2                             YATES   
Network                                            Unnamed: 2_level_1 Unnamed: 2_level_2                         NETWORK 5   
SQRP Total Points Earned                           Unnamed: 3_level_1 Unnamed: 3_level_2                               3.2   
SY 2015-2016 SQRP Rating                           Unnamed: 4_level_1 Unnamed: 4_level_2                          Level 2+   
...                                                                                                                    ...   
2015-2016 SQRP Individual Indicator Scores (Bas... Other Indicators   My Voice, My School 5 Essentials Survey.1          5   
                                                                      My Voice, My Sc

To unpack the multi-tier column headings, we will look to collapse the levels and then make column names more meaningful.

In [5]:
df_15_16.columns = df_15_16.columns.to_flat_index()

In [6]:
df_15_16.tail().T

,476,477,478,479,480
"(School ID, Unnamed: 0_level_1, Unnamed: 0_level_2)",609977,610345,610234,610235,609973
"(School Name, Unnamed: 1_level_1, Unnamed: 1_level_2)",WOODLAWN,WOODSON,YATES,YOUNG ES,ZAPATA
"(Network , Unnamed: 2_level_1, Unnamed: 2_level_2)",NETWORK 9,NETWORK 9,NETWORK 5,NETWORK 3,ISP
"(SQRP Total Points Earned, Unnamed: 3_level_1, Unnamed: 3_level_2)",2.6,2,3.2,2.6,3.4
"(SY 2015-2016 SQRP Rating, Unnamed: 4_level_1, Unnamed: 4_level_2)",Level 2,Level 2,Level 2+,Level 2,Level 2+
...,...,...,...,...,...
"(2015-2016 SQRP Individual Indicator Scores (Based on SY 2014-2015 Data), Other Indicators, My Voice, My School 5 Essentials Survey.1)",5,4,5,4,5
"(2015-2016 SQRP Individual Indicator Scores (Based on SY 2014-2015 Data), Other Indicators, My Voice, My School 5 Essentials Survey.2)",10,10,10,10,10
"(2015-2016 SQRP Individual Indicator Scores (Based on SY 2014-2015 Data), Other Indicators, Data Quality Index Score)",100,99.5,99.9,99.8,98.5
"(2015-2016 SQRP Individual Indicator Scores (Based on SY 2014-2015 Data), Other Indicators, Data Quality Index Score.1)",5,5,5,5,4


In [7]:
#set new column headings - purge spaces, make sublists to loop over tier 3 subheadings
lead_column_list = ['School_ID', 'School_Name', 'Network', 'SQRP_Score', 'SQRP_Rating', 'SQRP_Accountability']

tier_2s_with_subheadings = [
    'NWEA_Reading_Growth_Percentile', 
    'NWEA_Math_Growth_Percentile', 
    'NWEA_Percent_Students_Exceeding_Avg_Growth',
    'African_American_Reading_Growth_Percentile', 
    'Hispanic_Reading_Growth_Percentile',
    'ELL_Reading_Growth_Percentile',
    'Diverse_Learner_Reading_Growth_Percentile',
    'African_American_Math_Growth_Percentile', 
    'Hispanic_Math_Growth_Percentile',
    'ELL_Math_Growth_Percentile',
    'Diverse_Learner_Math_Growth_Percentile',
    'National_School_Attainment_Reading_Percentile_Gr3-8',
    'National_School_Attainment_Math_Percentile_Gr3-8',
    'National_School_Attainment_Reading_Percentile_Gr2',
    'National_School_Attainment_Math_Percentile_Gr2',
    'Percent_Students_ACCESS_Adequate_Progress'
    ]

tier_2s_with_short_subheadings = [
    'Average_Daily_Attendance_Rate_K-8',
    'My_Voice_My_School_Survey',
    'Data_Quality_Index'
    ]

tier_3_label_list = ['Score','Participation_Rate','Points','Weight']
tier_3_short_label_list = ['Score','Points','Weight']

#generate column heading list
column_list = []

#append lead col names
column_list.extend(lead_column_list)

#append four-part tier 3 headings
for heading in tier_2s_with_subheadings:
    for subheading in tier_3_label_list:
        output = heading + '_' + subheading
        column_list.append(output)

#append three-part tier 3 headings
for heading in tier_2s_with_short_subheadings:
    for subheading in tier_3_short_label_list:
        output = heading + '_' + subheading
        column_list.append(output)   
        

In [8]:
#verify list matches # of cols in df
len(df_15_16.columns) == len(column_list)

True

In [9]:
#import new column labels
df_15_16.columns = column_list

In [10]:
#check output
display_all(df_15_16.tail().T)

,476,477,478,479,480
School_ID,609977,610345,610234,610235,609973
School_Name,WOODLAWN,WOODSON,YATES,YOUNG ES,ZAPATA
Network,NETWORK 9,NETWORK 9,NETWORK 5,NETWORK 3,ISP
SQRP_Score,2.6,2,3.2,2.6,3.4
SQRP_Rating,Level 2,Level 2,Level 2+,Level 2,Level 2+
SQRP_Accountability,Provisional Support,Intensive Support,Good Standing,Provisional Support,Good Standing
NWEA_Reading_Growth_Percentile_Score,17,1,14,29,25
NWEA_Reading_Growth_Percentile_Participation_Rate,100,98,100,100,100
NWEA_Reading_Growth_Percentile_Points,2,1,2,2,2
NWEA_Reading_Growth_Percentile_Weight,18.75,18.75,13.75,13.75,13.75


Lastly, since we have removed all column references to school year being measured here, we will add in a new school year column to the DataFrame.

In [11]:
#add school year column
df_15_16['School_Year'] = '2015-2016'

We can now repeat this process over each of the school years within our time range. The 2016-17 dataset using identical column headings to the 2015-16 dataset, so we need only leverage the previous column names to convert the multi-tier Excel headings into pandas single-index column headings.

In [12]:
df_16_17 = pd.read_excel(f'{PATH}/{sy16_17_doc}',sheet_name = sheet_name, header = [0,1,2], na_values ='.')

In [13]:
display_all(df_16_17.tail().T)

473  \
2016-2017 SQRP Individual Indicator Scores (Bas... School ID                                          Unnamed: 0_level_2         609977   
                                                   School Name                                        Unnamed: 1_level_2       WOODLAWN   
                                                   Network                                            Unnamed: 2_level_2      NETWORK 9   
                                                   SQRP Total Points Earned                           Unnamed: 3_level_2            3.5   
                                                   SY 2016-2017 SQRP Rating                           Unnamed: 4_level_2        Level 1   
                                                   SY 2016-2017 Accountability Status                 Unnamed: 5_level_2  Good Standing   
NWEA MAP Growth Indicators - All Students          National School Growth Percentile - Reading        Score                          83   
                                                                                                      Part. Rate                     99   
                                                                                                      Points                          4   
                                                                                                      Weight                      18.75   
                                                   National School Growth Percentile - Math           Score                          50   
                                                                                                      Part. Rate                     99   
                                                                                                      Points                          3   
                                                                                                      Weight                      18.75   
                                                   % of Students Meeting/Exceeding National Ave Gr... Score                        58.1   
                                                                                                      Part. Rate                     99   
                                                                                                      Points                          3   
                                                                                                      Weight                         10   
NWEA MAP Growth Indicators - Priority Groups       African-American Growth Percentile - Reading       Score                          83   
                                                                                                      Part. Rate                     99   
                                                                                                      Points                          5   
                                                                                                      Weight                       1.25   
                                                   Hispanic Growth Percentile - Reading               Score                         NaN   
                                                                                                      Part. Rate                    NaN   
                                                                                                      Points                        NaN   
                                                                                                      Weight                        NaN   
                                                   English Learner Growth Percentile - Reading        Score                         NaN   
                                                                                                      Part. Rate                    NaN   
                                                                                                     

Since we need to load the Excel with a Multi-index, we first slice the columns of interest, then collapse the indexing.

In [14]:
#import new column headings
df_16_17.columns = df_16_17.columns.to_flat_index()
df_16_17.columns = column_list

In [15]:
#confirm output
display_all(df_16_17.tail().T)

,473,474,475,476,477
School_ID,609977,610345,610234,610235,609973
School_Name,WOODLAWN,WOODSON,YATES,YOUNG ES,ZAPATA
Network,NETWORK 9,NETWORK 9,NETWORK 5,NETWORK 3,ISP
SQRP_Score,3.5,3.5,3.7,3.6,4.4
SQRP_Rating,Level 1,Level 1,Level 1,Level 1,Level 1+
SQRP_Accountability,Good Standing,Intensive Support,Good Standing,Good Standing,Good Standing
NWEA_Reading_Growth_Percentile_Score,83,94,86,95,78
NWEA_Reading_Growth_Percentile_Participation_Rate,99,98,98,100,100
NWEA_Reading_Growth_Percentile_Points,4,5,4,5,4
NWEA_Reading_Growth_Percentile_Weight,18.75,17.5,13.75,13.75,13.75


With the data now placed into aligned headings, we add in the school year, and combine the two years processed so far into a single DataFrame.

In [16]:
#add in school year
df_16_17['School_Year'] = '2016-2017'

#combine dataframes
df = df_15_16.append(df_16_17, ignore_index=True)

In [17]:
display_all(df.tail().T)

,954,955,956,957,958
School_ID,609977,610345,610234,610235,609973
School_Name,WOODLAWN,WOODSON,YATES,YOUNG ES,ZAPATA
Network,NETWORK 9,NETWORK 9,NETWORK 5,NETWORK 3,ISP
SQRP_Score,3.5,3.5,3.7,3.6,4.4
SQRP_Rating,Level 1,Level 1,Level 1,Level 1,Level 1+
SQRP_Accountability,Good Standing,Intensive Support,Good Standing,Good Standing,Good Standing
NWEA_Reading_Growth_Percentile_Score,83,94,86,95,78
NWEA_Reading_Growth_Percentile_Participation_Rate,99,98,98,100,100
NWEA_Reading_Growth_Percentile_Points,4,5,4,5,4
NWEA_Reading_Growth_Percentile_Weight,18.75,17.5,13.75,13.75,13.75


We now move to the next school year. The 2017-2018 dataset no longer presents ACCESS score data, so we will modify our column heading list to match this minor change in formatting.

In [18]:
df_17_18 = pd.read_excel(f'{PATH}/{sy17_18_doc}',sheet_name = sheet_name, header = [0,1,2], na_values ='.')

In [19]:
display_all(df_17_18.tail().T)

468  \
SQRP SY2018 Individual Indicator Scores (Based ... School ID                                          Unnamed: 0_level_2          610542   
                                                   School Name                                        Unnamed: 1_level_2      WEST RIDGE   
                                                   Network                                            Unnamed: 2_level_2       Network 2   
                                                   SQRP Total Points Earned                           Unnamed: 3_level_2             4.2   
                                                   SY 2017-2018 SQRP Rating                           Unnamed: 4_level_2        Level 1+   
                                                   SY 2017-2018 Accountability Status                 Unnamed: 5_level_2   Good Standing   
NWEA MAP Growth Indicators - All Students          National School Growth Percentile - Reading        Score                           83   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           4   
                                                                                                      Weight                       16.25   
                                                   National School Growth Percentile - Math           Score                           86   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           4   
                                                                                                      Weight                       16.25   
                                                   Percent of Students Meeting/Exceeding National ... Score                         68.5   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           4   
                                                                                                      Weight                          10   
NWEA MAP Growth Indicators - Priority Groups       African-American Growth Percentile - Reading       Score                          NaN   
                                                                                                      Part. Rate                     NaN   
                                                                                                      Points                         NaN   
                                                                                                      Weight                         NaN   
                                                   Hispanic Growth Percentile - Reading               Score                           91   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           5   
                                                                                                      Weight                        1.25   
                                                   English Learner Growth Percentile - Reading        Score                           89   
                                                                                                      Part. Rate                     100   
                                                                         

In [20]:
#set new column headings - purge spaces, make sublists to loop over tier 3 subheadings
lead_column_list = ['School_ID', 'School_Name', 'Network', 'SQRP_Score', 'SQRP_Rating', 'SQRP_Accountability']

tier_2s_with_subheadings = [
    'NWEA_Reading_Growth_Percentile', 
    'NWEA_Math_Growth_Percentile', 
    'NWEA_Percent_Students_Exceeding_Avg_Growth',
    'African_American_Reading_Growth_Percentile', 
    'Hispanic_Reading_Growth_Percentile',
    'ELL_Reading_Growth_Percentile',
    'Diverse_Learner_Reading_Growth_Percentile',
    'African_American_Math_Growth_Percentile', 
    'Hispanic_Math_Growth_Percentile',
    'ELL_Math_Growth_Percentile',
    'Diverse_Learner_Math_Growth_Percentile',
    'National_School_Attainment_Reading_Percentile_Gr3-8',
    'National_School_Attainment_Math_Percentile_Gr3-8',
    'National_School_Attainment_Reading_Percentile_Gr2',
    'National_School_Attainment_Math_Percentile_Gr2'
    #,'Percent_Students_ACCESS_Adequate_Progress'
    ]

tier_2s_with_short_subheadings = [
    'Average_Daily_Attendance_Rate_K-8',
    'My_Voice_My_School_Survey',
    'Data_Quality_Index'
    ]

tier_3_label_list = ['Score','Participation_Rate','Points','Weight']
tier_3_short_label_list = ['Score','Points','Weight']

#generate column heading list
column_list = []

#append lead col names
column_list.extend(lead_column_list)

#append four-part tier 3 headings
for heading in tier_2s_with_subheadings:
    for subheading in tier_3_label_list:
        output = heading + '_' + subheading
        column_list.append(output)

#append three-part tier 3 headings
for heading in tier_2s_with_short_subheadings:
    for subheading in tier_3_short_label_list:
        output = heading + '_' + subheading
        column_list.append(output)   
        

In [21]:
#confirm that column list matches # of cols in df
len(df_17_18.columns) == len(column_list)

True

In [22]:
#update column names
df_17_18.columns = column_list

In [23]:
display_all(df_17_18.tail().T)

,468,469,470,471,472
School_ID,610542,610544,610548,610559,610586
School_Name,WEST RIDGE,AZUELA,STEM,SHIELDS MIDDLE,SOUTHEAST
Network,Network 2,ISP,Network 6,Network 8,Network 13
SQRP_Score,4.2,3.8,3.8,3.5,3.2
SQRP_Rating,Level 1+,Level 1,Level 1+,Level 1,Level 2+
SQRP_Accountability,Good Standing,Good Standing,Good Standing,Good Standing,Good Standing
NWEA_Reading_Growth_Percentile_Score,83,78,56,82,76
NWEA_Reading_Growth_Percentile_Participation_Rate,100,100,100,100,100
NWEA_Reading_Growth_Percentile_Points,4,4,3,4,4
NWEA_Reading_Growth_Percentile_Weight,16.25,16.25,17.5,17.5,18.75


With the data now aligned, we will add a school year column, and then append this year's data to the output DataFrame.

In [24]:
#add school year data
df_17_18['School_Year'] = '2017-2018'

#append to output df
df = df.append(df_17_18, ignore_index=True, sort = True)

In [25]:
display_all(df.tail().T)

,1427,1428,1429,1430,1431
African_American_Math_Growth_Percentile_Participation_Rate,NaN,NaN,100,NaN,NaN
African_American_Math_Growth_Percentile_Points,NaN,NaN,3,NaN,NaN
African_American_Math_Growth_Percentile_Score,NaN,NaN,40,NaN,NaN
African_American_Math_Growth_Percentile_Weight,NaN,NaN,1.25,NaN,NaN
African_American_Reading_Growth_Percentile_Participation_Rate,NaN,NaN,100,NaN,NaN
African_American_Reading_Growth_Percentile_Points,NaN,NaN,3,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,NaN,36,NaN,NaN
African_American_Reading_Growth_Percentile_Weight,NaN,NaN,1.25,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Points,4,4,5,4,3
Average_Daily_Attendance_Rate_K-8_Score,95.6,95.1,96.8,95.6,94.8


We now move into the next school year.  The 2018-2019 column headings are identical to the school year prior, so we need not regenerate the column heading list as we prepare the data.

In [26]:
df_18_19 = pd.read_excel(f'{PATH}/{sy18_19_doc}',sheet_name = sheet_name, header = [0,1,2], na_values ='.')

In [27]:
display_all(df_18_19.tail().T)

468  \
SQRP SY2019 Individual Indicator Scores (Based ... School ID                                          Unnamed: 0_level_2                610544   
                                                   School Name                                        Unnamed: 1_level_2                AZUELA   
                                                   Network                                            Unnamed: 2_level_2                   ISP   
                                                   SQRP Total Points Earned                           Unnamed: 3_level_2                   3.5   
                                                   SY 2018-2019 SQRP Rating                           Unnamed: 4_level_2               Level 1   
                                                   SY 2018-2019 Accountability Status                 Unnamed: 5_level_2         Good Standing   
NWEA MAP Growth Indicators - All Students          National School Growth Percentile - Reading        Score                                 64   
                                                                                                      Part. Rate                           100   
                                                                                                      Points                                 3   
                                                                                                      Weight                             16.25   
                                                   National School Growth Percentile - Math           Score                                 70   
                                                                                                      Part. Rate                           100   
                                                                                                      Points                                 4   
                                                                                                      Weight                             16.25   
                                                   Percent of Students Meeting/Exceeding National ... Score                               58.8   
                                                                                                      Part. Rate                           100   
                                                                                                      Points                                 3   
                                                                                                      Weight                                10   
NWEA MAP Growth Indicators - Priority Groups       African-American Growth Percentile - Reading       Score                                NaN   
                                                                                                      Part. Rate                           NaN   
                                                                                                      Points                               NaN   
                                                                                                      Weight                               NaN   
                                                   Hispanic Growth Percentile - Reading               Score                                 66   
                                                                                                      Part. Rate                           100   
                                                                                                      Points                                 4   
                                                                                                      Weight                              1.25   
                                                   English Learner Growth Percentile - Reading        Score                                 66   
                                                   

In [28]:
#update column names
df_18_19.columns = column_list

In [29]:
display_all(df_18_19.tail().T)

,468,469,470,471,472
School_ID,610544,610548,610559,610586,610588
School_Name,AZUELA,STEM,SHIELDS MIDDLE,SOUTHEAST,RICHARDSON
Network,ISP,ISP,Network 8,Network 13,Network 10
SQRP_Score,3.5,4,3.5,3.5,3.4
SQRP_Rating,Level 1,Level 1+,Level 1,Level 1,Level 2+
SQRP_Accountability,Good Standing,Good Standing,Good Standing,Good Standing,Good Standing
NWEA_Reading_Growth_Percentile_Score,64,52,40,52,62
NWEA_Reading_Growth_Percentile_Participation_Rate,100,100,100,100,100
NWEA_Reading_Growth_Percentile_Points,3,3,3,3,3
NWEA_Reading_Growth_Percentile_Weight,16.25,17.5,16.25,17.5,16.25


In [30]:
#add school year data
df_18_19['School_Year'] = '2018-2019'

#append to output df
df = df.append(df_18_19, ignore_index=True, sort = True)

In [31]:
display_all(df.tail().T)

,1900,1901,1902,1903,1904
African_American_Math_Growth_Percentile_Participation_Rate,NaN,100,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Points,NaN,3,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Score,NaN,40,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Weight,NaN,1.25,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Participation_Rate,NaN,100,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Points,NaN,4,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,58,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Weight,NaN,1.25,NaN,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Points,4,5,4,4,4
Average_Daily_Attendance_Rate_K-8_Score,95.5,96.5,95.6,95.7,95.5


We now move into the next - and final - school year.  This dataset now adds the ACCESS score data back in, so we will revert to the prior column heading set.

In [32]:
df_19_20 = pd.read_excel(f'{PATH}/{sy19_20_doc}',sheet_name = sheet_name, header = [0,1,2], na_values ='.')

In [33]:
display_all(df_19_20.tail().T)

468  \
SQRP SY2020 Individual Indicator Scores (Based ... School ID                                          Unnamed: 0_level_2          610559   
                                                   School Name                                        Unnamed: 1_level_2  SHIELDS MIDDLE   
                                                   Network                                            Unnamed: 2_level_2       Network 8   
                                                   SQRP Total Points Earned                           Unnamed: 3_level_2             3.7   
                                                   SY 2019-2020 SQRP Rating                           Unnamed: 4_level_2         Level 1   
                                                   SY 2019-2020 Accountability Status                 Unnamed: 5_level_2   Good Standing   
NWEA MAP Growth Indicators - All Students          National School Growth Percentile - Reading        Score                           83   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           4   
                                                                                                      Weight                       13.75   
                                                   National School Growth Percentile - Math           Score                           55   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           3   
                                                                                                      Weight                       13.75   
                                                   Percent of Students Meeting/Exceeding National ... Score                         58.5   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           3   
                                                                                                      Weight                          10   
NWEA MAP Growth Indicators - Priority Groups       African-American Growth Percentile - Reading       Score                          NaN   
                                                                                                      Part. Rate                     NaN   
                                                                                                      Points                         NaN   
                                                                                                      Weight                         NaN   
                                                   Hispanic Growth Percentile - Reading               Score                           83   
                                                                                                      Part. Rate                     100   
                                                                                                      Points                           5   
                                                                                                      Weight                        1.25   
                                                   English Learner Growth Percentile - Reading        Score                           79   
                                                                                                      Part. Rate                     100   
                                                                         

In [34]:
#set new column headings - purge spaces, make sublists to loop over tier 3 subheadings
lead_column_list = ['School_ID', 'School_Name', 'Network', 'SQRP_Score', 'SQRP_Rating', 'SQRP_Accountability']

tier_2s_with_subheadings = [
    'NWEA_Reading_Growth_Percentile', 
    'NWEA_Math_Growth_Percentile', 
    'NWEA_Percent_Students_Exceeding_Avg_Growth',
    'African_American_Reading_Growth_Percentile', 
    'Hispanic_Reading_Growth_Percentile',
    'ELL_Reading_Growth_Percentile',
    'Diverse_Learner_Reading_Growth_Percentile',
    'African_American_Math_Growth_Percentile', 
    'Hispanic_Math_Growth_Percentile',
    'ELL_Math_Growth_Percentile',
    'Diverse_Learner_Math_Growth_Percentile',
    'National_School_Attainment_Reading_Percentile_Gr3-8',
    'National_School_Attainment_Math_Percentile_Gr3-8',
    'National_School_Attainment_Reading_Percentile_Gr2',
    'National_School_Attainment_Math_Percentile_Gr2',
    'Percent_Students_ACCESS_Adequate_Progress'
    ]

tier_2s_with_short_subheadings = [
    'Average_Daily_Attendance_Rate_K-8',
    'My_Voice_My_School_Survey',
    'Data_Quality_Index'
    ]

tier_3_label_list = ['Score','Participation_Rate','Points','Weight']
tier_3_short_label_list = ['Score','Points','Weight']

#generate column heading list
column_list = []

#append lead col names
column_list.extend(lead_column_list)

#append four-part tier 3 headings
for heading in tier_2s_with_subheadings:
    for subheading in tier_3_label_list:
        output = heading + '_' + subheading
        column_list.append(output)

#append three-part tier 3 headings
for heading in tier_2s_with_short_subheadings:
    for subheading in tier_3_short_label_list:
        output = heading + '_' + subheading
        column_list.append(output)   
        

In [35]:
#confirm number of col names in list matches df
len(df_19_20.columns) == len(column_list)

True

In [36]:
#import new col names
df_19_20.columns = column_list

In [37]:
#add school year col
df_19_20['School_Year'] = '2019-2020'

#append data to main df
df = df.append(df_19_20, ignore_index = True, sort = True)

In [38]:
display_all(df.tail().T)

,2373,2374,2375,2376,2377
African_American_Math_Growth_Percentile_Participation_Rate,NaN,NaN,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Points,NaN,NaN,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Weight,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Participation_Rate,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Points,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Weight,NaN,NaN,NaN,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Points,4,4,4,5,5
Average_Daily_Attendance_Rate_K-8_Score,95.5,95,95.2,96.9,97.6


## Data Cleaning

With the raw data imported and combined into a single DataFrame, we can now clean the combined DataFrame and prepare it for export and analysis.

First, we will begin by force-typing all numericals to ensure both the data can be exported cleanly and so that it will be suitable for analysis.

In [39]:
display_all(df.dtypes)

African_American_Math_Growth_Percentile_Participation_Rate                 object
African_American_Math_Growth_Percentile_Points                             object
African_American_Math_Growth_Percentile_Score                              object
African_American_Math_Growth_Percentile_Weight                             object
African_American_Reading_Growth_Percentile_Participation_Rate              object
African_American_Reading_Growth_Percentile_Points                          object
African_American_Reading_Growth_Percentile_Score                           object
African_American_Reading_Growth_Percentile_Weight                          object
Average_Daily_Attendance_Rate_K-8_Points                                   object
Average_Daily_Attendance_Rate_K-8_Score                                    object
Average_Daily_Attendance_Rate_K-8_Weight                                   object
Data_Quality_Index_Points                                                  object
Data_Quality_Ind

We will build a new list of columns that can be typed into `float64`, and then force typing for each of those columns.

In [40]:
#set new column headings - purge spaces, make sublists to loop over tier 3 subheadings
lead_column_list = ['School_ID', 'SQRP_Score']

tier_2s_with_subheadings = [
    'NWEA_Reading_Growth_Percentile', 
    'NWEA_Math_Growth_Percentile', 
    'NWEA_Percent_Students_Exceeding_Avg_Growth',
    'African_American_Reading_Growth_Percentile', 
    'Hispanic_Reading_Growth_Percentile',
    'ELL_Reading_Growth_Percentile',
    'Diverse_Learner_Reading_Growth_Percentile',
    'African_American_Math_Growth_Percentile', 
    'Hispanic_Math_Growth_Percentile',
    'ELL_Math_Growth_Percentile',
    'Diverse_Learner_Math_Growth_Percentile',
    'National_School_Attainment_Reading_Percentile_Gr3-8',
    'National_School_Attainment_Math_Percentile_Gr3-8',
    'National_School_Attainment_Reading_Percentile_Gr2',
    'National_School_Attainment_Math_Percentile_Gr2',
    'Percent_Students_ACCESS_Adequate_Progress'
    ]

tier_2s_with_short_subheadings = [
    'Average_Daily_Attendance_Rate_K-8',
    'Data_Quality_Index'
    ]

tier_3_label_list = ['Score','Participation_Rate','Points','Weight']
tier_3_short_label_list = ['Score','Points','Weight']

#generate column heading list
column_list = []

#append lead col names
column_list.extend(lead_column_list)

#append four-part tier 3 headings
for heading in tier_2s_with_subheadings:
    for subheading in tier_3_label_list:
        output = heading + '_' + subheading
        column_list.append(output)

#append three-part tier 3 headings
for heading in tier_2s_with_short_subheadings:
    for subheading in tier_3_short_label_list:
        output = heading + '_' + subheading
        column_list.append(output)   
        

In [41]:
#drop column of leftover heading data
df.drop(0, axis = 0, inplace = True)

In [42]:
#change column typing to float as appropriate
for column in column_list:
    try:
        df[column] = df[column].astype('float64')
    except:
        print(f'column {column} could not be typed')
        

In [43]:
#force typing for two cols excluded in the above
df['My_Voice_My_School_Survey_Points'] = df['My_Voice_My_School_Survey_Points'].astype('float64')
df['My_Voice_My_School_Survey_Weight'] = df['My_Voice_My_School_Survey_Weight'].astype('float64')

#change typing of school ID to int
df['School_ID'] = df['School_ID'].astype('int')

In [44]:
display_all(df.dtypes)

African_American_Math_Growth_Percentile_Participation_Rate                float64
African_American_Math_Growth_Percentile_Points                            float64
African_American_Math_Growth_Percentile_Score                             float64
African_American_Math_Growth_Percentile_Weight                            float64
African_American_Reading_Growth_Percentile_Participation_Rate             float64
African_American_Reading_Growth_Percentile_Points                         float64
African_American_Reading_Growth_Percentile_Score                          float64
African_American_Reading_Growth_Percentile_Weight                         float64
Average_Daily_Attendance_Rate_K-8_Points                                  float64
Average_Daily_Attendance_Rate_K-8_Score                                   float64
Average_Daily_Attendance_Rate_K-8_Weight                                  float64
Data_Quality_Index_Points                                                 float64
Data_Quality_Ind

At this point, the full dataset is ready for use - I will export the whole DataFrame as a pickle so that it may be imported again later without losing the formatting.

In [45]:
df.to_pickle(f'{PATH}/joined_dataframe.pkl')

To reduce the overall number of columns to something more manageable, we will export the 'weight' data to a seperate table, save that table as a csv file, and then drop the columns from the main DataFrame.

In [46]:
column_list.append('My_Voice_My_School_Survey_Points')
column_list.append('My_Voice_My_School_Survey_Score')
column_list.append('My_Voice_My_School_Survey_Weight')

In [47]:
#initialize list with primary keys for data
weight_cols = ['School_ID', 'School_Year']

#add col values to list
for column in column_list:
    if 'Weight' in column:
        weight_cols.append(column)
        

In [48]:
#subset data, save to csv
weight_df = df.loc[:,weight_cols]
weight_df.to_csv(f'{OUTPUT_PATH}/sqrp_weights.csv', index = False)

In [49]:
#pop key values from column list
weight_cols.remove('School_ID')
weight_cols.remove('School_Year')

#remove weight cols from df
df.drop(weight_cols, axis = 1, inplace = True)

In [50]:
display_all(df.tail().T)

,2373,2374,2375,2376,2377
African_American_Math_Growth_Percentile_Participation_Rate,NaN,NaN,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Points,NaN,NaN,NaN,NaN,NaN
African_American_Math_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Participation_Rate,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Points,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Points,4,4,4,5,5
Average_Daily_Attendance_Rate_K-8_Score,95.5,95,95.2,96.9,97.6
Data_Quality_Index_Points,5,5,5,5,5
Data_Quality_Index_Score,99.5,99.7,99.8,99.9,100


We will repeat the same approach for the 'points' and 'rate' columns: we will seperate those columns into a new DataFrame, export them to csv, then drop the columns from the DataFrame.

In [51]:
#initialize list with primary keys for data
points_cols = ['School_ID', 'School_Year']

#add col values to list
for column in column_list:
    if 'Points' in column:
        points_cols.append(column)
        

In [52]:
#subset data, save to csv
score_df = df.loc[:,points_cols]
score_df.to_csv(f'{OUTPUT_PATH}/sqrp_points.csv', index = False)

In [53]:
#pop key values from column list
points_cols.remove('School_ID')
points_cols.remove('School_Year')

#remove weight cols from df
df.drop(points_cols, axis = 1, inplace = True)

In [54]:
points_cols

['NWEA_Reading_Growth_Percentile_Points',
 'NWEA_Math_Growth_Percentile_Points',
 'NWEA_Percent_Students_Exceeding_Avg_Growth_Points',
 'African_American_Reading_Growth_Percentile_Points',
 'Hispanic_Reading_Growth_Percentile_Points',
 'ELL_Reading_Growth_Percentile_Points',
 'Diverse_Learner_Reading_Growth_Percentile_Points',
 'African_American_Math_Growth_Percentile_Points',
 'Hispanic_Math_Growth_Percentile_Points',
 'ELL_Math_Growth_Percentile_Points',
 'Diverse_Learner_Math_Growth_Percentile_Points',
 'National_School_Attainment_Reading_Percentile_Gr3-8_Points',
 'National_School_Attainment_Math_Percentile_Gr3-8_Points',
 'National_School_Attainment_Reading_Percentile_Gr2_Points',
 'National_School_Attainment_Math_Percentile_Gr2_Points',
 'Percent_Students_ACCESS_Adequate_Progress_Points',
 'Average_Daily_Attendance_Rate_K-8_Points',
 'Data_Quality_Index_Points',
 'My_Voice_My_School_Survey_Points']

Repeating for 'rate' columns...

In [55]:
#initialize list with primary keys for data
rates_cols = ['School_ID', 'School_Year']

#add col values to list
for column in column_list:
    if 'Rate' in column:
        #exclude Attendance cols
        if not 'Attendance' in column:
            rates_cols.append(column)

In [56]:
rates_cols

['School_ID',
 'School_Year',
 'NWEA_Reading_Growth_Percentile_Participation_Rate',
 'NWEA_Math_Growth_Percentile_Participation_Rate',
 'NWEA_Percent_Students_Exceeding_Avg_Growth_Participation_Rate',
 'African_American_Reading_Growth_Percentile_Participation_Rate',
 'Hispanic_Reading_Growth_Percentile_Participation_Rate',
 'ELL_Reading_Growth_Percentile_Participation_Rate',
 'Diverse_Learner_Reading_Growth_Percentile_Participation_Rate',
 'African_American_Math_Growth_Percentile_Participation_Rate',
 'Hispanic_Math_Growth_Percentile_Participation_Rate',
 'ELL_Math_Growth_Percentile_Participation_Rate',
 'Diverse_Learner_Math_Growth_Percentile_Participation_Rate',
 'National_School_Attainment_Reading_Percentile_Gr3-8_Participation_Rate',
 'National_School_Attainment_Math_Percentile_Gr3-8_Participation_Rate',
 'National_School_Attainment_Reading_Percentile_Gr2_Participation_Rate',
 'National_School_Attainment_Math_Percentile_Gr2_Participation_Rate',
 'Percent_Students_ACCESS_Adequate_Pr

In [57]:
#subset data, save to csv
score_df = df.loc[:,rates_cols]
score_df.to_csv(f'{OUTPUT_PATH}/sqrp_rates.csv', index = False)

In [58]:
#pop key values from column list
rates_cols.remove('School_ID')
rates_cols.remove('School_Year')

#remove weight cols from df
df.drop(rates_cols, axis = 1, inplace = True)

In [59]:
display_all(df.tail().T)

,2373,2374,2375,2376,2377
African_American_Math_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Score,95.5,95,95.2,96.9,97.6
Data_Quality_Index_Score,99.5,99.7,99.8,99.9,100
Diverse_Learner_Math_Growth_Percentile_Score,99,96,69,NaN,NaN
Diverse_Learner_Reading_Growth_Percentile_Score,98,87,86,NaN,NaN
ELL_Math_Growth_Percentile_Score,63,NaN,28,NaN,NaN
ELL_Reading_Growth_Percentile_Score,79,NaN,60,NaN,NaN
Hispanic_Math_Growth_Percentile_Score,53,88,77,NaN,NaN
Hispanic_Reading_Growth_Percentile_Score,83,91,78,NaN,NaN


In [60]:
df.columns

Index(['African_American_Math_Growth_Percentile_Score',
       'African_American_Reading_Growth_Percentile_Score',
       'Average_Daily_Attendance_Rate_K-8_Score', 'Data_Quality_Index_Score',
       'Diverse_Learner_Math_Growth_Percentile_Score',
       'Diverse_Learner_Reading_Growth_Percentile_Score',
       'ELL_Math_Growth_Percentile_Score',
       'ELL_Reading_Growth_Percentile_Score',
       'Hispanic_Math_Growth_Percentile_Score',
       'Hispanic_Reading_Growth_Percentile_Score',
       'My_Voice_My_School_Survey_Score', 'NWEA_Math_Growth_Percentile_Score',
       'NWEA_Percent_Students_Exceeding_Avg_Growth_Score',
       'NWEA_Reading_Growth_Percentile_Score',
       'National_School_Attainment_Math_Percentile_Gr2_Score',
       'National_School_Attainment_Math_Percentile_Gr3-8_Score',
       'National_School_Attainment_Reading_Percentile_Gr2_Score',
       'National_School_Attainment_Reading_Percentile_Gr3-8_Score', 'Network',
       'Percent_Students_ACCESS_Adequate_Progres

In [61]:
output_list_cols = [
    'School_ID',  
    'School_Year',
    'School_Name', 
    'Network',
    'SQRP_Rating', 
    'SQRP_Score',
    'SQRP_Accountability',
    'African_American_Math_Growth_Percentile_Score',
    'African_American_Reading_Growth_Percentile_Score',
    'Average_Daily_Attendance_Rate_K-8_Score', 
    'Diverse_Learner_Math_Growth_Percentile_Score',
    'Diverse_Learner_Reading_Growth_Percentile_Score',
    'ELL_Math_Growth_Percentile_Score',
    'ELL_Reading_Growth_Percentile_Score',
    'Hispanic_Math_Growth_Percentile_Score',
    'Hispanic_Reading_Growth_Percentile_Score',
    'My_Voice_My_School_Survey_Score', 
    'NWEA_Math_Growth_Percentile_Score',
    'NWEA_Percent_Students_Exceeding_Avg_Growth_Score',
    'NWEA_Reading_Growth_Percentile_Score',
    'National_School_Attainment_Math_Percentile_Gr2_Score',
    'National_School_Attainment_Math_Percentile_Gr3-8_Score',
    'National_School_Attainment_Reading_Percentile_Gr2_Score',
    'National_School_Attainment_Reading_Percentile_Gr3-8_Score', 
    'Percent_Students_ACCESS_Adequate_Progress_Score',
    'Data_Quality_Index_Score'
    ]

output = df.loc[:,output_list_cols]

In [62]:
display_all(output.tail().T)

,2373,2374,2375,2376,2377
School_ID,610559,610586,610588,610589,610590
School_Year,2019-2020,2019-2020,2019-2020,2019-2020,2019-2020
School_Name,SHIELDS MIDDLE,SADLOWSKI,RICHARDSON,SOR JUANA,BRONZEVILLE CLASSICAL
Network,Network 8,Network 13,Network 10,Network 8,Network 9
SQRP_Rating,Level 1,Level 1+,Level 1+,Inability to Rate,Inability to Rate
SQRP_Score,3.7,4.1,4,NaN,NaN
SQRP_Accountability,Good Standing,Good Standing,Good Standing,Good Standing,Good Standing
African_American_Math_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
African_American_Reading_Growth_Percentile_Score,NaN,NaN,NaN,NaN,NaN
Average_Daily_Attendance_Rate_K-8_Score,95.5,95,95.2,96.9,97.6


In [63]:
output.to_csv(f'{OUTPUT_PATH}/school_ids_and_sqrp_scores.csv',index = False)

In [ ]:
bcp 
    cps_report_card.dbo.geocoding 
    in 'C:\Users\cahib\Documents\Code\CPS_Report_Card\SQL\school_locations.csv' 
    -F 2  #start line of input file
    -S learn-edfi.database.windows.net #server name
    -U steve #username
    
bcp geocoding in 'C:\Users\cahib\Documents\Code\CPS_Report_Card\SQL\school_locations.csv' -F 2 -S learn-edfi.database.windows.net -d cps_report_card -U steve

bcp geocoding in 'C:\temp\school_locations.csv' -F 2 -S learn-edfi.database.windows.net -d cps_report_card -U steve

In [ ]:
bcp geocoding in C:\Users\cahib\Documents\Code\CPS_Report_Card\SQL\school_locations.csv -S learn-edfi.database.windows.net -d cps_report_card -U steve -q -c -t  ,

## Data Subsetting for LEARN Network

To filter LEARN schools, we will use the School IDs, which are as follows:

* 400046 - Romano Butler
* 400047 - Campbell
* 400048 - Excel
* 400107 - South Chicago
* 400111 - Perkins
* 400151 - LEARN 7
* 400165 - LEARN Middle School

In [ ]:
id_list = ['400046', '400047', '400048', '400107', '400111', '400151', '400165']
learn_df = df[df['School ID'].isin(id_list)]

#push raw output to csv file
learn_df.to_csv(f'{PATH}/LEARN SQRP Ratings 2015-2020.csv', index = False)

## Comparison Data Gathering

To generate the most accurate possible apples-to-apples comparison, we will compare each of our Chicago schools against four schools in the same or nearby neighborhoods.

* Romano Butler
    - Johnson Elementary (AUSL) - 610274
    - Faraday Elementary (Network 5) - 610055
    - Gregory Math & Science Academy (Network 5) - 609954
    - Lawdale Elementary Community Academy (Network 5) - 610034
    
* Excel
    - Cather Elementary (Network 5) - 610251
    - Ward L Elementary (Network 5) - 610133
    - Morton Elementary Career Academy (AUSL) - 610257
    - Gregory Math & Science Academy (Network 5) - 609954

* Hunter-Perkins
    - Joplin (Network 11) - 609805
    - Barton (Network 11) - 609790
    - Foster Park (Network 11) - 609927
    - Cook (Network 11) - 609864

* Campbell
    - Cather Elementary (Network 5) - 610251
    - Dett (Network 6) - 610252
    - Spencer Technology Academy (Network 3) - 610183
    - Faraday (Network 5) - 610055

* South Chicago
    - Thorp J N (Network 12) - 610200
    - Mireles (Network 12) - 610171
    - Powell (Network 12) - 610281
    - Burnham (Network 13) - 609821
    
* LEARN 7 
    - Leland (Network 3) - 610305
    - Spencer (Network 3) - 610183
    - Ellington (Network 3) - 609904
    - Ward L Elementary (Network 5) - 610133

* LEARN Middle School
    - Cather Elementary (Network 5) - 610251
    - Ward L Elementary (Network 5) - 610133
    - Ellington (Network 3) - 609904
    - Beidler (Network 5) - 609797
    
* LEARN 9 Waukegan
    - Oakdale
    - Whittier
    - North
    - Glenwood
    - Carman-Buckner

* LEARN 6 & 10 North Chicago
    - D187 - Neal Math and Science
    - Forrestal
    - Katzenmaier 
    - Evelyn Alexander

In [ ]:
romano_butler_comps = [400046, 610274, 610055, 609954, 610034]
excel_comps = [400048, 610251, 610133, 610257, 609954]
hunter_perkins_comps = [400111, 609805, 609790, 609927, 609864]
campbell_comps = [400047, 610251, 610252, 610183, 610055]
south_chicago_comps = [400107, 610200, 610171, 610281, 609821]
learn_seven_comps = [400151, 610305, 610183, 609904, 610133]
learn_middle_comps = [400165, 610305, 610183, 609904, 610133]

In [ ]:
#reduce year entry to second year in entry, force typing
df['School Year'] = df['School Year'].apply(lambda x: re.split('-',x)[0])
df['School Year'] = df['School Year'].astype('int')

In [ ]:
df1 = df[df['School ID'].isin(romano_butler_comps)].copy()
df2 = df[df['School ID'].isin(excel_comps)].copy()
df3 = df[df['School ID'].isin(hunter_perkins_comps)].copy()
df4 = df[df['School ID'].isin(campbell_comps)].copy()
df5 = df[df['School ID'].isin(south_chicago_comps)].copy()
df6 = df[df['School ID'].isin(learn_seven_comps)].copy()
df7 = df[df['School ID'].isin(learn_middle_comps)].copy()

We will now write a function that will allow us to generate each sheet of the output Excel file programmatically.  This function will create a pivot of the data with schools as rows, years as columns, and SQRP ratings values; it will also generate a line graph representing the same data.

In [ ]:
def generate_graph_color(school, idx_count):
    '''
    Returns LEARN purple if the school variable contains 'LEARN',
    otherwise returns colors within same shade for other schools
    '''
    gray_dict = {
        2: '#D3D3D3', #lightgray
        3: '#A9A9A9', #darkgray
        4: '#696969', #dimgray
        5: '#708090', #slategray
        6: '#D3D3D3' #lightgray
    }
    if 'LEARN' in school:
        return "#542D81"
    
    else:
        return gray_dict[idx_count]    
    

In [ ]:
##manual creation of a test spreadsheet

# with pd.ExcelWriter(f'{PATH}/pivot_test.xlsx') as writer:
#     output = pd.pivot(to_graph, index = 'School Name', values = 'SQRP Total Points Earned', columns = 'School Year')
    
#     sheet_name = 'test'
#     output.to_excel(writer, sheet_name=sheet_name)
    
#     workbook = writer.book
#     worksheet = writer.sheets[sheet_name]
    
#     #initialize chart
#     chart = workbook.add_chart({'type': 'line'})
    
#     #establish starting col in Excel output
#     col = 2
    
#     #generate lines for graph
#     for school in output.index:
#         chart.add_series({
#             'name': school,
#             'categories': f'={sheet_name}!$B$1:$F$1',
#             'values': f'={sheet_name}!$B${col}:$F${col}' ,
#             'line':       {'color': generate_graph_color(school, col)},
#             })
#         col += 1
    
#     #set axis values
#     chart.set_x_axis({'name': 'Year'})
#     chart.set_y_axis({
#         'name': 'SQPR Rating', 
#         'major_gridlines': {'visible': False},
#         'min': 1.5    
#         })
    
#     #set size of chart in pixels
#     chart.set_size({'width': 720, 'height': 576})
    
#     #add graph to sheet
#     worksheet.insert_chart('I3', chart)

In [ ]:
def make_sqrp_excel_sheet_with_graph(df, sheet_name):
    '''
    Generates a Pivot of SQRP ratings and a 
    line graph within an active ExcelWriter session.
    '''
    sheet_name_ref = f"'{sheet_name}'"
    output = pd.pivot(df, index = 'School Name', values = 'SQRP Total Points Earned', columns = 'School Year')
    
    output.to_excel(writer, sheet_name=sheet_name)
    
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    #initialize chart
    chart = workbook.add_chart({'type': 'line'})
    
    #establish starting col in Excel output
    col = 2
    
    #generate lines for graph
    for school in output.index:
        chart.add_series({
            'name': school,
            'categories': f'={sheet_name_ref}!$B$1:$F$1',
            'values': f'={sheet_name_ref}!$B${col}:$F${col}' ,
            'line':       {'color': generate_graph_color(school, col)},
            })
        col += 1
    
    #set axis values
    chart.set_x_axis({'name': 'Year'})
    chart.set_y_axis({
        'name': 'SQPR Rating', 
        'major_gridlines': {'visible': False},
        'min': 1.5    
        })
    
    #set size of chart in pixels
    chart.set_size({'width': 720, 'height': 576})
    
    #add graph to sheet
    worksheet.insert_chart('I3', chart)

In [ ]:
with pd.ExcelWriter(f'{PATH}/SQRP Rating Comps, 2015-2020.xlsx') as writer:
    make_sqrp_excel_sheet_with_graph(df1, 'Romano Butler')
    make_sqrp_excel_sheet_with_graph(df2, 'Excel')
    make_sqrp_excel_sheet_with_graph(df3, 'Hunter Perkins')
    make_sqrp_excel_sheet_with_graph(df4, 'Campbell')
    make_sqrp_excel_sheet_with_graph(df5, 'South Chicago')
    make_sqrp_excel_sheet_with_graph(df6, 'LEARN 7')
    make_sqrp_excel_sheet_with_graph(df7, 'LEARN Middle School')

Here is code to graph the data within Python

In [ ]:
to_graph = df[df['School ID'].isin(hunter_perkins_comps)].copy()

# #reduce year entry to second year in entry, force typing
to_graph['School Year'] = to_graph['School Year'].apply(lambda x: re.split('-',x)[0])
to_graph['School Year'] = to_graph['School Year'].astype('int')

palette ={
    "LEARN - PERKINS":"#542D81",
    "FOSTER PARK":"#7F7F7F",
    "COOK":"#7F7F7F",
    "JOPLIN":"#7F7F7F",
    "BARTON":"#7F7F7F"
    }

fig, ax = plt.subplots(1, 1, figsize = (6, 6), dpi=300)

ax = sns.lineplot(x="School Year", y="SQRP Total Points Earned", hue = 'School Name', data=to_graph)

